# Tutorial 5: Engine-specific implementation

In this tutorial, we will discuss how to add an [engine-specific implementation](https://eagerx.readthedocs.io/en/master/guide/api_reference/object/index.html#eagerx.core.entities.Object.example_bridge)
for an object.

The following will be covered:
- Adding an [engine-specific implementation](https://eagerx.readthedocs.io/en/master/guide/api_reference/object/index.html#eagerx.core.entities.Object.example_bridge)
- Initializing the corresponding [bridge](https://eagerx.readthedocs.io/en/master/guide/api_reference/bridge/index.html)
- Train with the newly added [engine-specific implementation](https://eagerx.readthedocs.io/en/master/guide/api_reference/object/index.html#eagerx.core.entities.Object.example_bridge)

In the remainder of this tutorial we will go more into detail on this concept.

Furthermore, at the end of this notebook you will find an exercise.
For the exercise you will have to add/modify a couple of lines of code, which are marked by

```python

# START EXERCISE [BLOCK_NUMBER]

# END EXERCISE [BLOCK_NUMBER]
```

## Pendulum Swing-up

We will assume that we already have the object definition of the underactuated pendulum that we used in the [first](https://colab.research.google.com/github/eager-dev/eagerx_tutorials/blob/master/tutorials/pendulum/1_environment_creation.ipynb) tutorial with its dynamics simulated by the ODE bridge and corresponding ordinary differential equations (ODEs).

Our goal is to interface the underactuated pendulum we used in the previous tutorials with a different physics-engine. This allows us to investigate the effect of the physics engine on the learned performance. 

## Notebook Setup

In order to be able to run the code, we need to install the *eagerx_tutorials* package and ROS.

In [1]:
try:
    import eagerx_tutorials
except ImportError:
    !{"echo 'Installing eagerx-tutorials with pip.' && pip install eagerx-tutorials  >> /tmp/eagerx_install.txt 2>&1"}
if 'google.colab' in str(get_ipython()):
  !{"curl 'https://raw.githubusercontent.com/eager-dev/eagerx_tutorials/master/scripts/setup_colab.sh' > ~/setup_colab.sh"}
  !{"bash ~/setup_colab.sh"}

# Setup interactive notebook
# Required in interactive notebooks only.
from eagerx_tutorials import helper
helper.setup_notebook()
env = None

# Allows reloading of registered entites from changed files
# Required in interactive notebooks only.
%reload_ext autoreload
%autoreload 1

Not running on CoLab.
Execute ROS commands as "!...".
ROS noetic available.


## Let's get started

We start by importing the required packages and initializing EAGERx.

In [2]:
import eagerx
import eagerx_tutorials.pendulum  # Registers Pendulum

# Initialize eagerx (starts roscore if not already started.)
eagerx.initialize("eagerx_core")

... logging to /home/r2ci/.ros/log/e01f8844-cc46-11ec-8a21-2d459256326b/roslaunch-r2ci-Alienware-m15-R4-38714.log
started roslaunch server http://127.0.0.1:46717/
ros_comm version 1.15.14


SUMMARY

PARAMETERS
 * /rosdistro: noetic
 * /rosversion: 1.15.14

NODES

[INFO] [1651737499.287495]: Roscore cannot run as another roscore/master is already running. Continuing without re-initializing the roscore.


We will again create an environment with the *Pendulum* object, like we did in the [first](https://colab.research.google.com/github/eager-dev/eagerx_tutorials/blob/master/tutorials/pendulum/1_environment_creation.ipynb) and [second](https://colab.research.google.com/github/eager-dev/eagerx_tutorials/blob/master/tutorials/pendulum/2_reset_and_step.ipynb) tutorials.

Let's make the *Pendulum* object and add it to an empty graph.

In [3]:
# Make the pendulum
# START EXERCISE 1.3
# Add "u" to the sensors.
pendulum = eagerx.Object.make("Pendulum", "pendulum", actuators=["u"], sensors=["theta", "dtheta", "image"], states=["model_state"])
# ENDXERCISE 1.3

# Define rate in Hz
rate = 30.0

# Initialize empty graph
graph = eagerx.Graph.create()

# Add pendulum to the graph
graph.add(pendulum)

# Connect the pendulum to an action and observation
graph.connect(action="voltage", target=pendulum.actuators.u)
graph.connect(source=pendulum.sensors.theta, observation="angle")
graph.connect(source=pendulum.sensors.dtheta, observation="angular_velocity")
# START EXERCISE 1.3
# Connect sensor u to an observation.
# END EXERCISE 1.3

# Render image
graph.render(source=pendulum.sensors.image, rate=rate)

In the preceding tutorials, we simulated the pendulum dynamics with a set of ordinairy differential equations (ODE). In other words, the ODEs acted as the physics-engine. However, in robotics we are ultimately interested in how our learned controller performs in the real-world. EAGERx views the real-world as just another physics-engine. Ideally, we would only want to swap-out those components that are specific to the ODE physics-engine with the real-world specific components. In this way, all other (agnostic) components, such as controllers, are exactly the same in both setups, thus minimizing the differences between our real-world experiments and ODE simulation.

In EAGERx, this is achieved by grouping the physics-engine specific components of an object into *engine-specific implementations*. The implementation is then registered to the bridge that corresponds to the interfaced physics-engine.

Thus, an object in EAGERx will have a **single** agnostic definition of the sensors, actuators, and states, but may have **multiple** *engine-specific implementations* for each physics-engine. This implementation then defines how the sensors, actuators, and states are simulated using a specified bridge that, in turn, corresponds to a physics-engine (e.g. Pybullet, real-world, Gazebo, ...).

Lets start by printing info on the pendulum.

In [4]:
eagerx.Object.info("Pendulum")

Registered entity_id=`Pendulum`:
   entity_type: `Object`
   module: `eagerx_tutorials.pendulum.objects`
   file: `/home/r2ci/eagerx-dev/eagerx_tutorials/eagerx_tutorials/pendulum/objects.py`

Supported bridges:
 - OdeBridge

Make this spec with (use `entity_id: str = "Pendulum"`):
   spec = Object.make(entity_id: str, name: str, actuators: List[str] = None, sensors: List[str] = None, states: List[str] = None, rate: float = 30.0, render_shape: List[int] = None)

class Pendulum(Object):
   spec(spec: eagerx.core.specs.ObjectSpec, name: str, actuators: List[str] = None, sensors: List[str] = None, states: List[str] = None, rate: float = 30.0, render_shape: List[int] = None):
      docs:
         Object spec of Pendulum

   agnostic(spec: eagerx.core.specs.ObjectSpec, rate: float):
      config:
       - render_shape: [480, 480]
      sensors:
       - theta: <class 'std_msgs.msg._Float32.Float32'>
       - dtheta: <class 'std_msgs.msg._Float32.Float32'>
       - image: <class 'sensor_msgs

The printed info shows, amongst other things, the supported bridges of the *Pendulum*. Currently, only the [OdeBridge](https://github.com/eager-dev/eagerx_ode/blob/master/eagerx_ode/bridge.py) is supported. 

In the exercise of this tutorial, we will finish the code below. This will add support for a physics-engine that is interfaced by the [GymBridge](https://github.com/eager-dev/eagerx/blob/master/eagerx/bridges/openai_gym/bridge.py). Basically, the physics-engine is the [Pendulum-v0](https://gym.openai.com/envs/Pendulum-v0/) environment and it uses the dynamics of the [Pendulum-v1](https://gym.openai.com/envs/Pendulum-v0/) environment to simulate our pendulum.

In essence, an *engine-specific* implementation must do the following:
- Set any engine-specific parameters that are required by the bridge to add an object to the physics-engine. These correspond to the parameters that have been registered with the [bridge.add_object](https://eagerx.readthedocs.io/en/master/guide/api_reference/bridge/index.html#eagerx.core.entities.Bridge.add_object). In the example below, we need to set the `env_id` to the id of the pendulum environment. You can get info on all the registered parameters (Also called `bridge_config`) with `eagerx.Bridge.info("GymBridge")` under the `add_object` section.
- Make [EngineNodes](https://eagerx.readthedocs.io/en/master/guide/api_reference/node/engine_node.html) and connect them in the [EngineGraph](https://eagerx.readthedocs.io/en/master/guide/api_reference/graph/engine_graph.html) to the object's registered sensors and actuators. It is not compulsory to connect all registered sensors and actuators. Hence, you may only implement a subset of them. In this example, we do not connect the registered sensor `u`, which is not a problem as long as we do not select it at run-time.
- Make [EngineStates](https://eagerx.readthedocs.io/en/master/guide/api_reference/engine_state/index.html) and set them to the object's registered states. In this example, we use dummy states, because OpenAI gym environments do not have a uniform API to set environment states. See the engine-implementation of the OdeBridge [here](https://github.com/eager-dev/eagerx_tutorials/blob/3ddc2eb7558c7825095611fec3a01a47f5e7af79/eagerx_tutorials/pendulum/objects.py#L108-L168) for a non-trivial example with engine states.


In [5]:
from eagerx import register
from eagerx.bridges.openai_gym.bridge import GymBridge
import eagerx.bridges.openai_gym  # Register openai engine-specific nodes (ObservationSensor, ActionActuator, GymImage)
import eagerx_tutorials.pendulum  # Register tutorial engine-specific nodes (FloatOutput)

eagerx.Bridge.info("GymBridge", "add_object")

# This decorator registers the engine-specific implementation for the entity_id="Pendulum".
@register.bridge("Pendulum", GymBridge)  
def gym_bridge(spec: eagerx.specs.ObjectSpec, graph: eagerx.EngineGraph):
    """Engine-specific implementation (GymBridge) of the Pendulum object."""
    
    # Set engine-specific parameters
    spec.GymBridge.env_id = "Pendulum-v1"
    
    # Create engine states that implement the registered states
    # Note: OpenAI environment do not have a uniform API for setting the API state after a .reset().
    #       Therefore, we use a DummyState that simply does nothing (so that it can still be selected, without raising an error).
    #       Though, you could create a bridge dedicated to the "Pendulum-v1" environment that does allow setting the state.
    #       We will not do that here. The same story holds for the dynamic parameters.
    spec.GymBridge.states.model_state = eagerx.EngineState.make("DummyState")
    spec.GymBridge.states.model_parameters = eagerx.EngineState.make("DummyState")
    
    # Create sensor engine nodes.
    image = eagerx.EngineNode.make("GymImage", "image", rate=spec.sensors.image.rate, shape=spec.config.render_shape)
    theta = eagerx.EngineNode.make("FloatOutput", "theta", rate=spec.sensors.theta.rate, idx=0)
    
    # Create engine node that implements the dtheta observation
    # START EXERCISE 1.1.a
    dtheta = eagerx.EngineNode.make("FloatOutput", "dtheta", rate=spec.sensors.dtheta.rate, idx=1)
    # END EXERCISE 1.1.a

    # Create actuator engine node
    action = eagerx.EngineNode.make("ActionActuator", "action", rate=spec.actuators.u.rate, process=2)

    # Use the observations produced by the "Pendulum-v1" to obtain theta and dtheta.
    # Because this observation is [sin(theta), cos(theta), dtheta], so we first convert it to [theta, dtheta]
    x = eagerx.EngineNode.make("ObservationSensor", "x", rate=spec.sensors.theta.rate, process=2)
    x.outputs.observation.converter = eagerx.Processor.make("Angle_DecomposedAngle", convert_to="theta_dtheta")
    
    # Add all engine nodes to the engine-specific graph
    graph.add([x, theta, image, action])
    
    # START EXERCISE 1.1.b
    # Add dtheta to graph
    graph.add(dtheta)
    # END EXERCISE 1.1.b
    
    # theta
    graph.connect(source=x.outputs.observation, target=theta.inputs.observation_array)
    graph.connect(source=theta.outputs.observation, sensor="theta")

    # dtheta
    # START EXERCISE 1.1.c
    # Connect dtheta here.
    graph.connect(source=x.outputs.observation, target=dtheta.inputs.observation_array)
    graph.connect(source=dtheta.outputs.observation, sensor="dtheta")
    # END EXERCISE 1.1.c
    
    # image
    graph.connect(source=image.outputs.image, sensor="image")

    # u
    # Note: not to be confused with sensor "u", for which we do not provide an implementation here.
    graph.connect(actuator="u", target=action.inputs.action)

Registered entity_id=`GymBridge`:
   entity_type: `Bridge`
   module: `eagerx.bridges.openai_gym.bridge`
   file: `/home/r2ci/.cache/pypoetry/virtualenvs/eagerx-tutorials-t4w5hBSU-py3.8/lib/python3.8/site-packages/eagerx/bridges/openai_gym/bridge.py`

Make this spec with (use `entity_id: str = "GymBridge"`):
   spec = Bridge.make(entity_id: str, rate, process: Union[int, NoneType] = 0, sync: Union[bool, NoneType] = True, real_time_factor: Union[float, NoneType] = 0, simulate_delays: Union[bool, NoneType] = True, log_level: Union[int, NoneType] = 40)

class GymBridge(Bridge):
   add_object(self, config, bridge_config, node_params, state_params):
      bridge_config:
       - env_id: None
      docs:

                 Adds an object whose dynamics are governed by a registered OpenAI gym environment.

                 :param config: The (agnostic) config of the :class:`~eagerx.core.entities.Object` that is to be added.
                 :param bridge_config: The bridge-specific config of t

After defining the engine-specific implementation for the GymBridge and registering it, we again print info on the pendulum. 

Now we see that the GymBridge **is** supported:

In [6]:
eagerx.Object.info("Pendulum")

Registered entity_id=`Pendulum`:
   entity_type: `Object`
   module: `eagerx_tutorials.pendulum.objects`
   file: `/home/r2ci/eagerx-dev/eagerx_tutorials/eagerx_tutorials/pendulum/objects.py`

Supported bridges:
 - OdeBridge
 - GymBridge

Make this spec with (use `entity_id: str = "Pendulum"`):
   spec = Object.make(entity_id: str, name: str, actuators: List[str] = None, sensors: List[str] = None, states: List[str] = None, rate: float = 30.0, render_shape: List[int] = None)

class Pendulum(Object):
   spec(spec: eagerx.core.specs.ObjectSpec, name: str, actuators: List[str] = None, sensors: List[str] = None, states: List[str] = None, rate: float = 30.0, render_shape: List[int] = None):
      docs:
         Object spec of Pendulum

   agnostic(spec: eagerx.core.specs.ObjectSpec, rate: float):
      config:
       - render_shape: [480, 480]
      sensors:
       - theta: <class 'std_msgs.msg._Float32.Float32'>
       - dtheta: <class 'std_msgs.msg._Float32.Float32'>
       - image: <class

We will start by using the OdeBridge similar to the previous tutorials. Later on in the exercises. we will switch and use the [GymBridge](https://github.com/eager-dev/eagerx/blob/master/eagerx/bridges/openai_gym/bridge.py) instead.

In [10]:
# Register both bridges
import eagerx_ode  # Registers OdeBridge
import eagerx.bridges.openai_gym  # Registers GymBridge

# Make the bridge
# START EXERCISE 1.2 & 1.5
bridge = eagerx.Bridge.make("OdeBridge", rate=rate)
# bridge = eagerx.Bridge.make("GymBridge", rate=rate, process=eagerx.process.ENVIRONMENT)
# END EXERCISE 1.2 & 1.5

At this point, we have create a graph containing the pendulum. We provide the graph to the environment together with the bridge. Based on this bridge, we will initialize the *engine-specific implementation* that was registered with this bridge. 
- If the [OdeBridge](https://github.com/eager-dev/eagerx_ode) is provided, we use the registered OdeBridge implementation [here](https://github.com/eager-dev/eagerx_tutorials/blob/3ddc2eb7558c7825095611fec3a01a47f5e7af79/eagerx_tutorials/pendulum/objects.py#L108-L168).
- If the [GymBridge](https://github.com/eager-dev/eagerx/blob/master/eagerx/bridges/openai_gym/bridge.py) is provided, we will use the registered implementation above.
- If we would have an implemention for the real-world and registered it with the [RealBridge](https://github.com/eager-dev/eagerx_reality/blob/m1aster/eagerx_reality/bridge.py), it would be selected when we choose the [RealBridge](https://github.com/eager-dev/eagerx_reality/blob/m1aster/eagerx_reality/bridge.py).

In [8]:
from typing import Dict
import numpy as np

# Define step function
def step_fn(prev_obs: Dict[str, np.ndarray], obs: Dict[str, np.ndarray], action: Dict[str, np.ndarray], steps: int):
    
    # Get angle and angular velocity
    # Take first element because of window size (covered in other tutorial)
    th = obs["angle"][0] 
        
    thdot = obs["angular_velocity"][0]
    
    # Convert from numpy array to float
    u = float(action["voltage"])
    
    # Calculate cost
    # Penalize angle error, angular velocity and input voltage
    cost = th**2 + 0.1 * thdot**2 + 0.001 * u**2  
    
    # Determine when is the episode over
    # currently just a timeout after 100 steps
    done = steps > 100
    
    # Set info, tell the algorithm the termination was due to a timeout
    # (the episode was truncated)
    info = {"TimeLimit.truncated": steps > 100}
    
    return obs, -cost, done, info

# Initialize Environment
env = eagerx.EagerxEnv(name="PendulumEnv", rate=rate, graph=graph, bridge=bridge, step_fn=step_fn)

[INFO] [1651737499.741226]: Pre-existing parameters under namespace "/PendulumEnv" deleted.
[INFO] [1651737499.770724]: Node "/PendulumEnv/env/supervisor" initialized.
[INFO] [1651737499.914192]: Node "/PendulumEnv/bridge" initialized.
[INFO] [1651737500.046074]: Node "/PendulumEnv/environment" initialized.
[INFO] [1651737500.069811]: Node "/PendulumEnv/env/render" initialized.
[INFO] [1651737500.150501]: Node "/PendulumEnv/pendulum/theta" initialized.
[INFO] [1651737500.172670]: Node "/PendulumEnv/pendulum/dtheta" initialized.


Finally, we train the agent using [Stable Baselines3](https://stable-baselines3.readthedocs.io/en/master/), again similar to the preceding tutorials.

In [9]:
import stable_baselines3 as sb
from eagerx.wrappers import Flatten

# Toggle render
env.render("human")

# Stable Baselines3 expects flattened actions & observations
# Convert observation and action space from Dict() to Box()
env = Flatten(env)

# Initialize learner
model = sb.SAC("MlpPolicy", env, verbose=1, device="cpu")

# Train for 1 minute (sim time)
model.learn(total_timesteps=int(60 * rate))

env.shutdown()

[INFO] [1651737500.232753]: Adding object "pendulum" of type "Pendulum" to the simulator.
[INFO] [1651737500.396211]: Node "/PendulumEnv/pendulum/x" initialized.
[INFO] [1651737500.413769]: Node "/PendulumEnv/pendulum/image" initialized.
[INFO] [1651737500.431491]: Node "/PendulumEnv/pendulum/pendulum_actuator" initialized.
[INFO] [1651737500.499870]: Node "/PendulumEnv/pendulum/u" initialized.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
[INFO] [1651737500.605191]: START RENDERING!
[INFO] [1651737500.606223]: [pendulum/image] START RENDERING!
[INFO] [1651737500.872738]: Nodes initialized.
[INFO] [1651737500.925152]: Pipelines initialized.


KeyboardInterrupt: 

# Exercise

In this exercise you will add a new engine-specific implementation to the object definition of the underactuated pendulum.

For this exercise, you will need to modify or add some lines of code in the cells above.
These lines are indicated by the following comments:

```python
# START EXERCISE [BLOCK_NUMBER]

# END EXERCISE [BLOCK_NUMBER]
```

However, feel free to play with the other code as well if you are interested.
We recommend you to restart and run all code after each section (in Colab there is the option *Restart and run all* under *Runtime*).

## 1. Add support for a new physics-engine
Up until now, we have simulated the pendulum dynamics with the *engine-specific* implementation [here](https://github.com/eager-dev/eagerx_tutorials/blob/3ddc2eb7558c7825095611fec3a01a47f5e7af79/eagerx_tutorials/pendulum/objects.py#L108-L168) that was registered with the [OdeBridge](https://github.com/eager-dev/eagerx_ode).

Most informative would be an exercise where we interface a real pendulum. Unfortunately, interactive notebooks do not allow us to easily demonstrate this without forcing users to have the exact same real pendulum we have in our lab. Therefore, we will instead add an implementation for the already defined OpenAI's [GymBridge](https://github.com/eager-dev/eagerx/blob/master/eagerx/bridges/openai_gym/bridge.py). We created [GymBridge](https://github.com/eager-dev/eagerx/blob/master/eagerx/bridges/openai_gym/bridge.py) so that any [OpenAI environment](https://gym.openai.com/envs/#classic_control) could be used as the physics-engine. In this exercise we will use the dynamics of the [Pendulum-v1](https://gym.openai.com/envs/Pendulum-v0/) environment to simulate our pendulum. For this, we will make use of the already defined engine nodes [here](https://github.com/eager-dev/eagerx/blob/master/eagerx/bridges/openai_gym/enginenodes.py).

Given that you've already created the engine nodes to interface the real pendulum, you can easily add an implementation for the [RealBridge](https://github.com/eager-dev/eagerx_reality/blob/m1aster/eagerx_reality/bridge.py) to train with a real pendulum following the same steps. Creating [engine nodes](https://eagerx.readthedocs.io/en/master/guide/api_reference/node/engine_node.html) is very similar to creating regular nodes which was already covered in [tutorial 4](https://colab.research.google.com/github/eager-dev/eagerx_tutorials/blob/master/tutorials/pendulum/4_nodes.ipynb). 

### Add your code to the following blocks: 

1.1.a Make an `EngineNode` that will be `dtheta`. Use `entity_id=FloatOutput` and set `idx=1`. *(hint: look at the code for `theta`).*  
1.1.b Add EngineNode `dtheta` to the engine graph. *(hint: look at the code for `theta`).*  
1.1.c Connect `dtheta` to the corresponding sensor with `sensor=dtheta`. *(hint: look at the code for `theta`).*  
1.2 Select the GymBridge by uncommenting the marked line. Run the code *(note: you may need to restart your kernel)*.  
1.3 Now, select sensor `u` (not to be mistaken with the actuator `u`!!) for the pendulum and connect it as an `observation`. Run the code and observe that it fails. As the error states, we did not provide an implementation for sensor `u`. This highlights that it is not compulsory to implement every actuator, sensor, or state that was defined by the object. You are free to only support a subset of them. However, you **will** get an error if you try to run with one that does not have an *engine-specific* implementation for the selected bridge.  
1.4 Switch back to using the OdeBridge (while still selecting sensor `u`). Run the code. It should again run without problems, as the OdeBridge **does** have an implementation for the sensor `u`.  
